As we saw earlier in the course, _algorithms_ are just processes for solving problems. We have already seen many algorithms that solved specific problems (such as computing prime numbers, factoring numbers, or computing Fibonacci numbers). But are those the only ways to solve those problems? In fact, there are many different ways of arriving at the same result, and some of those ways are better than others (by better, we generally mean either faster, simpler, or both).

There are many different strategies for coming up with algorithms and improving algorithms, but here are a few big ideas:
1. Don't do the same calculation multiple times.
2. Don't do unnecessary computation (where the result is either obvious or irrelevant).
3. Divide and conquer.

The first idea is one that we looked at in the last chapter using caching. Caching helps to keep us from repeating computations we have already done.

The second idea comes into play a lot when we are using loops or recursion. Which cases are either simple (so we can skip them and just use the result we already know) or meaningless? For example, if I'm searching for prime numbers, I don't have to check the even numbers, because other than $2$, there are no even primes.

The third idea is a completely new idea for this section, which is why it gets its own memo:

**Memo: Divide and Conquer**  
_Divide and Conquer_ is a general strategy for solving large problems by breaking them into smaller (and hopefully easier) sub-problems. On many problems, this produces faster and simpler algorithms than approaches that attempt to solve the entire problem in one pass.

**Recipe: Merge Sort**  
Given a list of numbers, how would you put them in order from least to greatest? One approach is to use recursion along with a divide-and-conquer approach. See below:

In [1]:
def merge_sort(list_of_numbers):
    if len(list_of_numbers) <= 1: # Base Case, list of one number are already sorted
        return list_of_numbers
    else:
        #Split into two lists
        split_point = len(list_of_numbers) // 2
        list1 = list_of_numbers[:split_point] # List up to, but not including split point
        list2 = list_of_numbers[split_point:] # list including split point and later numbers
        
        #Sort each list
        list1 = merge_sort(list1)
        list2 = merge_sort(list2)
        
        #Merge the two lists
        return merge(list1, list2)

def merge(list1, list2):
    new_list = []
    index1 = 0
    index2 = 0
    while(len(new_list) < len(list1) + len(list2)):
        if index1 >= len(list1):
            new_list.append(list2[index2])
            index2 = index2 + 1
        elif index2 >= len(list2):
            new_list.append(list1[index1])
            index1 = index1 + 1
        elif list1[index1] < list2[index2]:
            new_list.append(list1[index1])
            index1 = index1 + 1
        else:
            new_list.append(list2[index2])
            index2 = index2 + 1
    return new_list

In [2]:
some_list = [2000, 1, 364, 50, 7]
print( merge_sort(some_list) )

[1, 7, 50, 364, 2000]


How can we apply these ideas to some of the things we have been computing using number theory? We've already alluded to ways to apply this to computing prime numbers, so let's outline how each of the pieces could be applied:

First, how can we avoid repeating computation? The simple answer is to store prime numbers as we compute them, so that when we go to compute new prime numbers, we don't have to start from scratch.

In [3]:
prime_numbers = [2, 3] # This stores the list of prime numbers so far

def prime_tester(number): # Only checks against primes we have computed so far.
    is_prime = True
    for prime in prime_numbers:
        if number % prime == 0:
            is_prime = False
    return is_prime

def compute_primes_until(number):
    start_number = prime_numbers[len(prime_numbers) - 1] + 1 # Starts at the last number computed + 1
    for test_number in range(start_number, number + 1):
        if prime_tester(test_number):
            prime_numbers.append(test_number)
    return prime_numbers

print(compute_primes_until(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


What about the second step? How can we eliminate unnecessary computations? Well, as mentioned in a previous chapter, we really only need to check the primes whose square is less than the number in question, and as mentioned above, we know all future primes will be odd numbers, so we can skip over the evens. We can use some new syntax to make it simpler to express this:

**Syntax: break**  
When used inside a loop, the "break" keyword tells Python to stop running the loop and move on to the code that follows it.

In [4]:
def prime_tester(number): # Only checks against primes we have computed so far.
    is_prime = True
    for prime in prime_numbers:
        if prime * prime > number:
            break
        if number % prime == 0:
            is_prime = False
    return is_prime

def compute_primes_until(number):
    test_number = prime_numbers[len(prime_numbers) - 1] + 1 # Starts at the last number computed + 1
    while test_number < number + 1:
        if prime_tester(test_number):
            prime_numbers.append(test_number)
        test_number = test_number + 2 # Skips all the even numbers
    return prime_numbers

print(compute_primes_until(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


Finally, how can we apply the third step? Is there any way to divide and conquer while computing the prime numbers? In some sense, we are working on sub-problems (easier pieces of the problem) by starting with finding primes up to a small number. But can we do better? It turns out that there is some really interesting math that could replace the prime_tester with a function that doesn't need a list of prime numbers. That algorithm is called the _AKS Primality Test_. Implementing it is more complicated than the level of the course right now, but that can be used to break up the subproblems (testing) more effectively for really big numbers.

For now, let's move on to thinking about prime factorization. How can we use the ideas here to make factoring numbers more efficient? Here's a basic factoring algorithm, like the one you might have written earlier in the course:

In [5]:
def factor(number):
    primes = compute_primes_until(number)
    factors = {} # Will map from a prime number to its multiplicity - how many times it is a factor
    for prime_index in range(len(primes)): # A shorthand for this in Python is `for prime in primes`
        prime = primes[prime_index]
        while number % prime == 0: # We'll divide by the prime until the remainder is not divisible by it
            factors[prime] = factors.get(prime, 0) + 1
            number = number / prime
    return factors

print(factor(60))

{2: 2, 3: 1, 5: 1}


How can we avoid repeating computation? A good first pass is to cache the result of the function, so if we try to compute the same factorization, the output will be already saved.

In [6]:
# This is just the cache function from last chapter:
def cache_function(function):
    the_cache = {} # A different copy is created every time cache_function is called/used.
    def lookup_first(input_number): # Define a new function that checks the cache first
        if input_number in the_cache:
            return the_cache[input_number]
        else:
            the_cache[input_number] = function(input_number)
            return the_cache[input_number]
    return lookup_first

factor = cache_function(factor)
print(factor(60))

{2: 2, 3: 1, 5: 1}


What about eliminating unnecessary computation?

**Problem: Skip the factorization if the number is prime**  
Edit the function definition for `factor` below so that if the number is prime, the factorization is output without doing much work.

In [7]:
def factor(number):
    primes = compute_primes_until(number)
    factors = {} # Will map from a prime number to its multiplicity - how many times it is a factor
    ### TODO: EDIT THIS FUNCTION SO THAT number ISN'T FACTORED IF IT'S PRIME.
    
    for prime_index in range(len(primes)): # A shorthand for this in Python is `for prime in primes`
        prime = primes[prime_index]
        while number % prime == 0: # We'll divide by the prime until the remainder is not divisible by it
            factors[prime] = factors.get(prime, 0) + 1
            number = number / prime
    return factors

factor = cache_function(factor)
print(factor(60))

{2: 2, 3: 1, 5: 1}


Finally, maybe there's some way to use divide-and-conquer to simplify the algorithm, or make it faster. What we should notice is that after we divide by a prime number, we're left with a new number we need to factor. This doesn't necessarily make our factorization faster for a single number, but it does make it a little simpler, and the savings from the caching add up if we factor many different numbers:

In [8]:
def factor(number):
    primes = compute_primes_until(number)
    factors = {} # Default case: 1 has no factors
    for prime_index in range(len(primes)): # A shorthand for this in Python is `for prime in primes`
        prime = primes[prime_index]
        if number % prime == 0: # We'll factor the remainder
            factors = factor(number // prime) # This is solving the remaining factorization problem.
            factors[prime] = factors.get(prime, 0) + 1
            break # All factors have been computed.
    return factors

factor = cache_function(factor)
print(factor(60))

{5: 1, 3: 1, 2: 2}


We have already seen how duplicate computations can be removed from computing Fibonacci numbers by using caching, and this has a huge impact on the speed with which we can compute Fibonacci Numbers. But are we doing any extra computations that we don't need to? To refresh your memory, here's the Fibonacci function:

In [9]:
def fibonacci(n):
    """Computes the nth Fibonacci number"""
    # This is a "Doc String" - Think of it as another kind of comment.
    
    if n <= 0:
        return 0
    if n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci = cache_function(fibonacci)
print( fibonacci(35) )

9227465


This function is pretty efficient (with caching) - it ends up computing all of the Fibonacci numbers between 0 and n exactly once. But is it possible that we don't need to compute all of the Fibonacci numbers up to n in order to compute the nth Fibonacci number? To answer this question, we're going to journey into some mathematics and see some interesting connections to binary numbers.

New mathematical insights are often generated by keeping track of extra information, or by asking new questions. For example, we could ask - how would the Fibonacci numbers change if the first two elements weren't 0 and 1?

**Problem: Fibonacci's from 5 and 8**  
Modify the above function to compute Fibonacci numbers if the starting numbers are 5 and 8. What do you notice about the sequence of the first 10 Fibonacci numbers starting from 0, 1 and starting from 5, 8?

**Problem: Fibonacci's from 10 and 10**  
Modify the above function to compute Fibonacci numbers where 10 and 10 are the starting numbers. What do you notice about the relationship between the Fibonacci numbers starting from 0, 1 and starting from 10, 10?

**Problem: Fibonacci's from 13 and -8**  
Modify the above function to compute Fibonacci numbers where 13 and -8 are the starting numbers. What do you notice about the relationship between the Fibonacci numbers starting from 0, 1 and starting from 13, -8?

(Doing these homework problems will help you understand the part that comes next)

How can we better understand the Fibonacci sequence? What if, instead of computing just one number, we keep track of extra information - how many times has the first Fibonacci number been added in, and how many times has the second been added in?

For example:  
$$F_0 = [1, 0]$$
$$F_1 = [0, 1]$$
$$F_2 = [1, 1]$$
$$F_3 = [1, 2]$$
$$F_4 = [2, 3]$$
$$F_5 = [3, 5]$$
$$...$$
$$F_n = [F_{n-1}[0] + F_{n-2}[0], F_{n-1}[1] + F_{n-2}[1]]$$

If you continue writing the pairs out like this, you may notice something very interesting - the components themselves take on values from the Fibonacci sequence. If you look at the recursive definition a little, it may become clear to you why this is so.

So what can we do with this? Well, we can compute shortcuts. If I want to compute $F_8$ and $F_9$, I can treat $F_4$ and $F_5$ as the starting elements and just apply the appropriate multipliers. 
$$F_8 = [F_4[0]*F_4[0] + F_4[1]*F_5[0], F_4[0]*F_4[1] + F_4[1]*F_5[1]] = [13, 21]$$
$$F_9 = [F_5[0]*F_4[0] + F_5[1]*F_5[0], F_5[0]*F_4[1] + F_5[1]*F_5[1]] = [21, 34]$$
To understand this better, let's consider the next step. We can use $F_8$ and $F_9$ to compute $F_{16}$ and $F_{17}$. And we can use $F_{16}$ and $F_{17}$ to compute $F_{32}$ and $F_{33}$. Notice a pattern? We are able to double the size of the Fibonacci numbers we can generate on each step. Now here's the connection to binary numbers: every number can be broken up into a sum of powers of two. So by computing just these Fibonacci numbers, we can figure out how to compute any Fibonacci number.

In [48]:
# Program for mapping an integer to a list of binary digits (biggest digits first)
def binary(n):
    if n == 0:
        return [0]
    elif n == 1:
        return [1]
    else:
        higher_digits = binary(n//2).copy() # copy the list so that caching isn't using the same object again and again.
        higher_digits.append(n%2)
        return higher_digits

binary = cache_function(binary)

# Caching Shortcuts and Using Them to Compute Fibonacci Numbers
def apply_shortcut(item_0, item_1, shortcut):
    return [shortcut[0]*item_0[0] + shortcut[1]*item_1[0], shortcut[0]*item_0[1] + shortcut[1]*item_1[1]]

def fibonacci_shortcuts(n):
    shortcuts = {0:[1, 0], 1:[0, 1], 2:[1, 1], 3:[1, 2]}
    
    if n in shortcuts:
        return shortcuts[n]
    else:
        zeros = 0
        ones = 0
        reversed_digits = list(reversed(binary(n)))
        # If it is a power of 2 or a power of 2 plus 1, generate the shortcut
        total_digits = sum(reversed_digits)
        if total_digits == 1:
            # It's a power of 2
            half = fibonacci_shortcuts(n//2)
            half_plus = fibonacci_shortcuts(n//2 + 1)
            return apply_shortcut(half, half_plus, half)
        elif total_digits ==2 and reversed_digits[0] == 1:
            # It's a power of 2 + 1
            half = fibonacci_shortcuts(n//2)
            half_plus = fibonacci_shortcuts(n//2 + 1)
            return apply_shortcut(half, half_plus, half_plus)
        else:
            # Add up the shortcuts.
            shortcut = fibonacci_shortcuts(reversed_digits[0])
            shortcut_plus = fibonacci_shortcuts(reversed_digits[0] + 1)
            for i in range(1, len(reversed_digits)):
                if reversed_digits[i] == 1:
                    new_shortcut = apply_shortcut(shortcut, shortcut_plus, fibonacci_shortcuts(2**i))
                    new_shortcut_plus = apply_shortcut(shortcut, shortcut_plus, fibonacci_shortcuts(2**i + 1))
                    shortcut = new_shortcut
                    shortcut_plus = new_shortcut_plus
            return shortcut
            
        # TODO... Actually Solve/Finish
    
    
fibonacci_shortcuts = cache_function(fibonacci_shortcuts)
    
def powers_fibonacci(n):
    # Returns the actual number computed using the shortcuts
    shortcut = fibonacci_shortcuts(n)
    return shortcut[1] # the same as 0 * shortcut[0] + 1 * shortcut[1]
    
    
powers_fibonacci = cache_function(powers_fibonacci)

In [51]:
#for i in range(17):
#    print(powers_fibonacci(i))
    
print(powers_fibonacci(35))

print(powers_fibonacci(10000)) # This is very fast.

9227465
33644764876431783266621612005107543310302148460680063906564769974680081442166662368155595513633734025582065332680836159373734790483865268263040892463056431887354544369559827491606602099884183933864652731300088830269235673613135117579297437854413752130520504347701602264758318906527890855154366159582987279682987510631200575428783453215515103870818298969791613127856265033195487140214287532698187962046936097879900350962302291026368131493195275630227837628441540360584402572114334961180023091208287046088923962328835461505776583271252546093591128203925285393434620904245248929403901706233888991085841065183173360437470737908552631764325733993712871937587746897479926305837065742830161637408969178426378624212835258112820516370298089332099905707920064367426202389783111470054074998459250360633560933883831923386783056136435351892133279732908133732642652633989763922723407882928177953580570993691049175470808931841056146322338217465637321248226383092103297701648054726243842374862411453093812206

**Problem: Fibonacci Pairs of Squares**  
Some fibonacci numbers can be written as the sum of 2 squares. For example, $2 = 1^2 + 1^2$, $5 = 1^2 + 2^2$, $13 = 2^2 + 3^2$, $34 = 3^2 + 5^2$, etc. Figure out which Fibonacci numbers can be written this way. Can you come up with a formula for how to break the nth Fibonacci number into a sum of squares (assuming it is one of the ones that can be)? Can you write a program to generate just the Fibonacci numbers that can be written this way?

**Problem: Even Fibonacci's**  
How often do even Fibonacci numbers show up? Can you write an efficient algorithm that computes only the even Fibonacci's (Make sure to eliminate duplicate computation and unnecessary computation - bonus points if you think of a way to apply _divide and conquer_.)

# Part 2: Euclid's Algorithm and the Chinese Remainder Theorem

Suppose that we want to find the largest factor that two numbers have in common. This is called the Greatest Common Divisor (GCD). One way the GCD could be computed is by factoring both numbers and finding the smallest multiplicity of each prime factor between the two of them.  This doesn't seem to be doing duplicate computation, but maybe it is doing unnecessary computation. Are there other ways of finding the GCD of two numbers?

That's exactly the question that Euclid (an ancient Greek mathematician) considered - and he made the following observation:  
If $a$ and $b$ are integers, and $g$ is the largest number such that  
$$a \equiv 0 \mod g$$  
and  
$$b \equiv 0 \mod g$$  
, then $a - b \equiv 0 \mod g$.  

Why is this useful? Well, we can reduce the size of the problem we are considering from finding the gcd(a, b) to finding the gcd(a-b, b). If we applied this multiple times in a row, we could even reduce the problem from gcd(a, b) to gcd(a%b, b)=gcd(b, a%b). We could apply this step repeatedly until one of the numbers divides the other, at which point we know it must be the greatest common divisor. This is the basis of Euclid's algorithm.

**Problem: Implement Euclid's Algorithm**  
Look up a description of Euclid's Algorithm and create an implementation in the cell below. Note: If you are doing it correctly, it should be a very short program (less than 10 lines of code). Make sure to include some test cases

In [52]:
# Provide your implementation of Euclid's Algorithm Here:
# ...


The Chinese Remainder Theorem also involves modular arithmetic, but uses it in a completely different way. It says that if I have two numbers, $a$ and $b$ and gcd(a, b)=1, then if I know that:
$$n \equiv q \mod a$$  
$$n \equiv p \mod b$$  
then there is a single number $m \leq a \times b$ such that  
$$m \equiv n \mod a \times b$$.

In other words, given the remainder of a number modulo two different numbers, I can find that number (provided it is less than the product of the two different numbers and the two different numbers don't have any prime factors in common).

As far as we know, this theorem was first stated by the Chinese mathematician Sunzi about 1800 years ago.

**Problem: Implement the Chinese Remainder Theorem**  
Although you are welcome to look up a description of the algorithm to find the number n given its remainders mod a and mod b, you are encouraged to think about how you would solve this problem on your own. You may find your own solution works quite well. Since this may take some time and effort, there are only two total problems in the homework for this part.

In [ ]:
# Implement an algorithm that finds the number known to exist because of the Chinese Remainder Theorem
# ...